# Pytest 实战教程（Notebook 版）

本 Notebook 将带你从零开始搭建一个使用 pytest 的项目，涵盖环境准备、首个测试、夹具、参数化、插件与覆盖率等主题。建议按顺序运行每个单元格，并在运行前确认你已在虚拟环境中安装 `pytest`。

## 1. 准备项目目录

我们先为示例项目创建目录结构：

- `demo_project/`：项目根目录
- `src/`：源代码目录
- `tests/`：测试目录
- `conftest.py`：保存共享 fixture 的位置

运行下方代码会在当前仓库里创建这一结构（如果目录已存在则跳过）。

In [ ]:
from pathlib import Path

project_root = Path('demo_project')
(project_root / 'src').mkdir(parents=True, exist_ok=True)
(project_root / 'tests').mkdir(parents=True, exist_ok=True)
print(project_root.resolve())

> ✅ 运行完毕后，将在当前目录中看到 `demo_project/`。接下来所有代码单元格都会默认以它为工作目录。

## 2. 编写被测代码

创建一个最简单的业务模块 `src/calculator.py`，其中提供 `add` 和 `divide` 两个函数，方便后续撰写不同类型的测试。

In [ ]:
from pathlib import Path

calculator_path = Path('demo_project/src/calculator.py')
calculator_path.write_text('def add(a: float, b: float) -> float:\n    """返回两数之和。\n\n    >>> add(1, 2)\n    3\n    """\n    return a + b\n\n\ndef divide(a: float, b: float) -> float:\n    """执行除法运算，如果除数为 0 会抛出 ZeroDivisionError。\n\n    >>> divide(6, 3)\n    2.0\n    """\n    return a / b\n', encoding='utf-8')
print(calculator_path.read_text(encoding='utf-8'))

## 3. 编写第一个测试

pytest 采用约定优于配置的方式，只需在 `tests/` 目录下新建以 `test_` 开头的文件即可自动发现。下方示例创建 `tests/test_calculator.py`，并包含基础断言。

In [ ]:
from pathlib import Path

test_path = Path('demo_project/tests/test_calculator.py')
test_path.write_text('from src.calculator import add, divide\n\n\nclass TestAdd:\n    def test_add_positive_numbers(self):\n        assert add(1, 2) == 3\n\n    def test_add_negative_numbers(self):\n        assert add(-1, -3) == -4\n\n\nclass TestDivide:\n    def test_divide_evenly(self):\n        assert divide(9, 3) == 3\n\n    def test_divide_by_zero_raises(self):\n        import pytest\n\n        with pytest.raises(ZeroDivisionError):\n            divide(1, 0)\n', encoding='utf-8')
print(test_path.read_text(encoding='utf-8'))

## 4. 运行测试

使用 `pytest` 命令即可运行全部测试。Notebook 中可以通过 `!` 或 `%%bash` 调用 Shell。

In [ ]:
%%bash
cd demo_project
pytest -q

如果看到 `4 passed`，说明基础环境正常。

## 5. 引入 fixture 复用准备逻辑

当测试需要重复的准备工作时，可以使用 fixture。下面创建 `tests/conftest.py`，并定义一个 `numbers` fixture，随后在测试中按参数名使用。

In [ ]:
from pathlib import Path

conftest_path = Path('demo_project/tests/conftest.py')
conftest_path.write_text('import pytest\n\n\n@pytest.fixture\ndef numbers():\n    return [1, 2, 3]\n', encoding='utf-8')
print(conftest_path.read_text(encoding='utf-8'))

为演示 fixture，向测试文件新增一个用例。

In [ ]:
from pathlib import Path

fixture_test_path = Path('demo_project/tests/test_with_fixture.py')
fixture_test_path.write_text('from src.calculator import add\n\n\ndef test_add_with_fixture(numbers):\n    assert add(numbers[0], numbers[1]) == 3\n', encoding='utf-8')
print(fixture_test_path.read_text(encoding='utf-8'))

再次运行 pytest，可以看到新增的测试用例。

In [ ]:
%%bash
cd demo_project
pytest tests/test_with_fixture.py -q

## 6. 参数化测试

使用 `@pytest.mark.parametrize` 可以让同一个测试在不同输入下重复运行。我们为 `add` 函数添加多个测试用例。

In [ ]:
from pathlib import Path

param_test_path = Path('demo_project/tests/test_parametrize.py')
param_test_path.write_text('import pytest\n\nfrom src.calculator import add\n\n\n@pytest.mark.parametrize("a,b,expected", [\n    (0, 0, 0),\n    (2, 5, 7),\n    (-2, 3, 1),\n])\ndef test_add_parametrized(a, b, expected):\n    assert add(a, b) == expected\n', encoding='utf-8')
print(param_test_path.read_text(encoding='utf-8'))

运行并查看 `-vv`（更详细的）输出。

In [ ]:
%%bash
cd demo_project
pytest tests/test_parametrize.py -vv

## 7. 标记、跳过与预期失败

pytest 支持为测试添加自定义标记，以及跳过或标记预期失败，帮助我们在 CI 中灵活控制。

In [ ]:
from pathlib import Path

marked_path = Path('demo_project/tests/test_marks.py')
marked_path.write_text('import sys\nimport pytest\n\nfrom src.calculator import divide\n\n\n@pytest.mark.skipif(sys.platform == "win32", reason="示例：Windows 暂不支持")\ndef test_skip_on_windows():\n    assert divide(4, 2) == 2\n\n\n@pytest.mark.xfail(reason="示例：待修复的缺陷")\ndef test_known_bug():\n    assert divide(1, 2) == 3\n', encoding='utf-8')
print(marked_path.read_text(encoding='utf-8'))

运行时使用 `-rxXs` 观察被跳过、预期失败等信息。

In [ ]:
%%bash
cd demo_project
pytest tests/test_marks.py -rxXs

## 8. 使用插件：pytest-cov 生成覆盖率

插件可以拓展 pytest 功能。以 `pytest-cov` 为例，先通过 `pip install pytest-cov` 安装，然后运行带 `--cov` 选项的测试。若本地尚未安装，可在 Notebook 中直接执行安装命令。

In [ ]:
# 如已安装 pytest-cov，可跳过此单元
%%bash
pip install pytest-cov >/tmp/pytest_cov_install.log && tail -n 5 /tmp/pytest_cov_install.log

In [ ]:
%%bash
cd demo_project
pytest --cov=src --cov-report=term-missing -q

输出中会显示各文件的覆盖率以及未被执行的行。

## 9. 在 CI 中运行 pytest

最后，我们给出一个 GitHub Actions 工作流示例，演示如何在持续集成环境中运行 pytest。

In [ ]:
from pathlib import Path

ci_path = Path('demo_project/.github/workflows/tests.yml')
ci_path.parent.mkdir(parents=True, exist_ok=True)
ci_path.write_text('name: Tests\n\non: [push, pull_request]\n\njobs:\n  test:\n    runs-on: ubuntu-latest\n    steps:\n      - uses: actions/checkout@v4\n      - uses: actions/setup-python@v5\n        with:\n          python-version: "3.11"\n      - run: pip install -r requirements.txt\n      - run: pytest -q\n', encoding='utf-8')
print(ci_path.read_text(encoding='utf-8'))

> 💡 在真实项目中，请根据需要补充依赖安装步骤或缓存策略。

## 10. 小结与下一步

通过本 Notebook，你已经：

1. 构建了一个最小可运行的 pytest 项目；
2. 学习了编写断言、使用 fixture、参数化测试、标记与跳过；
3. 体验了 pytest-cov 插件与覆盖率报告；
4. 了解了如何在 CI 中集成 pytest。

建议继续探索以下主题：

- 自定义 fixture 作用域与自动使用；
- 利用 `conftest.py` 组织更多共享逻辑；
- 编写插件或 hook 扩展 pytest 行为；
- 在真实项目中结合 Mock、数据库、API 等复杂场景编写测试。